In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
key_path = "./creds/service_acc_auth.json"
credentials = service_account.Credentials.from_service_account_file(key_path, scopes = ["https://www.googleapis.com/auth/cloud-platform"])
bigquery_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [2]:
# query new data <= 1/1
union_weeks = """
SELECT * , 
FROM `scanning-database.scanning_data_backup.W35`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W36`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W37`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W38`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W39`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W40`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W41`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W42`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W43`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W44`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W45`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W46`
"""

In [3]:
# Overwriting old PowerBI source data
job_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10,
destination = 'scanning-database.PowerBI.PowerBI',
write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
) # set query limit to 10G
query_job = bigquery_client.query(union_weeks, job_config)  # API request
rows = query_job.result()  # Waits for query to finish
data_in_use = rows.to_dataframe()

In [4]:
data_in_use.tail()

,Start_Time,Finish_Time,Activity,Seq_Code,Recipe_Name,Break_Reasons,Missing_Ingredients,Kitting_Line,Assembly_Batch,Event_Shift,Team_Leader,Pickers_Count,Time_Consumption,Week
97040,2020-12-10 11:36:00+00:00,2020-12-10 11:41:00+00:00,Production,0004 of 0006,Q4,None,None,KL19,Tuesday Assembly,Thursday Morning,iXUsr_Siti Rahmah Siregar 6092;iXUsr_Siti Rahm...,4,5.0,2020-42
97041,2020-12-10 11:53:00+00:00,2020-12-10 11:53:00+00:00,Production,0006 of 0006,Q4,None,None,KL19,Tuesday Assembly,Thursday Morning,iXUsr_Siti Rahmah Siregar 6092;iXUsr_Siti Rahm...,4,0.0,2020-42
97042,2020-12-10 11:31:00+00:00,2020-12-10 11:36:00+00:00,Production,0003 of 0006,Q4,None,None,KL19,Tuesday Assembly,Thursday Morning,iXUsr_Siti Rahmah Siregar 6092;iXUsr_Siti Rahm...,4,5.0,2020-42
97043,2020-12-10 11:41:00+00:00,2020-12-10 11:47:00+00:00,Production,0005 of 0006,Q4,None,None,KL19,Tuesday Assembly,Thursday Morning,iXUsr_Siti Rahmah Siregar 6092;iXUsr_Siti Rahm...,4,6.0,2020-42
97044,2020-12-10 11:29:00+00:00,2020-12-10 11:31:00+00:00,Production,0002 of 0006,Q4,None,None,KL19,Tuesday Assembly,Thursday Morning,iXUsr_Siti Rahmah Siregar 6092;iXUsr_Siti Rahm...,4,2.0,2020-42


In [5]:
data_in_use.shape

(97045, 14)